<a href="https://colab.research.google.com/github/HwangHanJae/BigDataPratice/blob/main/working_type1_naver_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [3]:
url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cur&date=20210503'

###Question 1

###네이버 영화 평점 순위의 랭킹사이트의 데이터를 requests를 이용하여 가져오고 parsing 하여 soup 변수에 저장하라

In [7]:
data = requests.get(url)
soup = bs(data.text)

soup.head()

[<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>,
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>,
 <meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="me2:image"/>,
 <meta content="네이버영화 " property="me2:post_tag"/>,
 <meta content="네이버영화" property="me2:category1"/>,
 <meta content="" property="me2:category2"/>,
 <meta content="랭킹 : 네이버 영화" property="og:title"/>,
 <meta content="영화, 영화인, 예매, 박스오피스 랭킹 정보 제공" property="og:description"/>,
 <meta content="article" property="og:type"/>,
 <meta content="https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=cur&amp;date=20210503" property="og:url"/>,
 <meta content="http://static.naver.net/m/movie/icons/OG_270_270.png" property="og:image"/>,
 <meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="og:article:thumbnailUrl"/>,
 <meta content="네이버 영화" property="og:article:author"/>,
 <meta content="https://movie.naver.com/" property="og:article:author:

###Question 2

###메타정보에 관한 모든 파싱값을 meta 변수에 저장하라

In [9]:
meta = soup.find_all("meta")
meta

[<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>,
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>,
 <meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="me2:image"/>,
 <meta content="네이버영화 " property="me2:post_tag"/>,
 <meta content="네이버영화" property="me2:category1"/>,
 <meta content="" property="me2:category2"/>,
 <meta content="랭킹 : 네이버 영화" property="og:title"/>,
 <meta content="영화, 영화인, 예매, 박스오피스 랭킹 정보 제공" property="og:description"/>,
 <meta content="article" property="og:type"/>,
 <meta content="https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=cur&amp;date=20210503" property="og:url"/>,
 <meta content="http://static.naver.net/m/movie/icons/OG_270_270.png" property="og:image"/>,
 <meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="og:article:thumbnailUrl"/>,
 <meta content="네이버 영화" property="og:article:author"/>,
 <meta content="https://movie.naver.com/" property="og:article:author:

###Question 3

###랭킹테이블의 모든파싱데이터를 가져와서 lstRanking 변수에 저장하라 class=list_ranking

In [16]:
lstRanking = soup.find(class_='list_ranking')
lstRanking.find(class_="blind")

<caption class="blind">랭킹 테이블</caption>

###Question 4

###lstRanking에서 모든 영화의 순위, 이름, 평점, 영화정보링크를 가져와서 데이터프레임을 만들고 movieTable 변수에 저장하라

In [49]:
#이름
title = [x.text.strip() for x in lstRanking.find_all("div", class_='tit5')]
#평점
point = [x.text for x in lstRanking.find_all("td", class_="point")]
#순위
rank = range(1, len(point)+1)
#영화정보링크
link = ["https://movie.naver.com" + x.a.get("href") for x in lstRanking.find_all('div', class_='tit5')]

columns = {"rank" : rank,
           "title" : title,
           "point" : point,
           "link" : link}
movieTable = pd.DataFrame(columns)
movieTable.head()

,rank,title,point,link
0,1,레옹,9.37,https://movie.naver.com/movie/bi/mi/basic.nave...
1,2,다시 태어나도 우리,9.35,https://movie.naver.com/movie/bi/mi/basic.nave...
2,3,피아니스트,9.33,https://movie.naver.com/movie/bi/mi/basic.nave...
3,4,"미안해요, 리키",9.32,https://movie.naver.com/movie/bi/mi/basic.nave...
4,5,부활: 그 증거,9.31,https://movie.naver.com/movie/bi/mi/basic.nave...


###Question 5

###movieTable에 date 컬럼을 추가하여 2021-05-03 을 입력하라

In [51]:
movieTable["date"] = pd.to_datetime("2021-05-03")

movieTable.head()

,rank,title,point,link,date
0,1,레옹,9.37,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-05-03
1,2,다시 태어나도 우리,9.35,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-05-03
2,3,피아니스트,9.33,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-05-03
3,4,"미안해요, 리키",9.32,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-05-03
4,5,부활: 그 증거,9.31,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-05-03


###Question 6

###21년 4월 1일 부터 4월 30일까지 일자별 리스트를 생성하여 하나의 데이터 프레임(변수명 total)에 저장하라. 날짜는 date컬럼에 생성한다.

In [54]:
def get_movieDataEachDays(day):

  if day<10:
    day = '0'+str(day)
  else:
    day = str(day)

  base_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cur&date=202104'+day
  data = requests.get(base_url)
  soup = bs(data.text)
  lstRanking = soup.find(class_="list_ranking")
  #이름
  title = [x.text.strip() for x in lstRanking.find_all("div", class_='tit5')]
  #평점
  point = [x.text for x in lstRanking.find_all("td", class_="point")]
  #순위
  rank = range(1, len(point)+1)
  #영화정보링크
  link = ["https://movie.naver.com" + x.a.get("href") for x in lstRanking.find_all('div', class_='tit5')]
  columns = {"rank" : rank,
           "title" : title,
           "point" : point,
           "link" : link}
  movieTable = pd.DataFrame(columns)
  movieTable["date"] = pd.to_datetime("2021-04-"+day)
  return movieTable

In [56]:
#날짜별 데이터프레임을 생성하고 최종적으로 하나의 데이터 프레임을 생성
dfLst = []

for day in range(1,31):
  df = get_movieDataEachDays(day)
  dfLst.append(df)

,rank,title,point,link,date
0,1,극장판 바이올렛 에버가든,9.48,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
1,2,가나의 혼인잔치: 언약,9.42,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
2,3,반지의 제왕: 왕의 귀환,9.38,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
3,4,죽은 시인의 사회,9.38,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
4,5,반지의 제왕: 두 개의 탑,9.36,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
...,...,...,...,...,...
45,46,팅커 테일러 솔저 스파이,8.21,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-30
46,47,테넷,8.17,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-30
47,48,문라이즈 킹덤,8.11,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-30
48,49,날씨의 아이,7.95,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-30


In [59]:
total = pd.concat(dfLst).reset_index(drop=True)
total.head()

,rank,title,point,link,date
0,1,극장판 바이올렛 에버가든,9.48,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
1,2,가나의 혼인잔치: 언약,9.42,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
2,3,반지의 제왕: 왕의 귀환,9.38,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
3,4,죽은 시인의 사회,9.38,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
4,5,반지의 제왕: 두 개의 탑,9.36,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01


###Question 7

###total 데이터에서 일자별 3위 rank의 데이터를 rankThree 변수에 저장하라

In [62]:
rankThree = total.loc[total["rank"] == 3].reset_index(drop=True)
rankThree.head()

,rank,title,point,link,date
0,3,반지의 제왕: 왕의 귀환,9.38,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-01
1,3,소년시절의 너,9.39,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-02
2,3,소년시절의 너,9.39,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-03
3,3,소년시절의 너,9.39,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-04
4,3,반지의 제왕: 왕의 귀환,9.38,https://movie.naver.com/movie/bi/mi/basic.nave...,2021-04-05


###Question 8

###해당기간동안 랭킹 1위 영화들의 그 횟수를 구하여라

In [65]:
total.loc[total["rank"] == 1].groupby(["title"]).sum()

,rank
title,
극장판 바이올렛 에버가든,22
레옹,2
부활: 그 증거,6


In [73]:
count = total.loc[total['rank'] == 1]["title"].value_counts().to_frame().rename(columns={"index" : "title","title":"count"})
count.head()

,count
극장판 바이올렛 에버가든,22
부활: 그 증거,6
레옹,2


###Question 9

###데이터를 재구성하여 일자별 랭킹의 영화 제목을 출력하라

In [84]:
rebulid = total.pivot(columns='date', index='rank',values='title')
rebulid.head()

date,2021-04-01,2021-04-02,2021-04-03,2021-04-04,2021-04-05,2021-04-06,2021-04-07,2021-04-08,2021-04-09,2021-04-10,2021-04-11,2021-04-12,2021-04-13,2021-04-14,2021-04-15,2021-04-16,2021-04-17,2021-04-18,2021-04-19,2021-04-20,2021-04-21,2021-04-22,2021-04-23,2021-04-24,2021-04-25,2021-04-26,2021-04-27,2021-04-28,2021-04-29,2021-04-30
rank,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,부활: 그 증거,부활: 그 증거,부활: 그 증거,부활: 그 증거,부활: 그 증거,부활: 그 증거,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,레옹,레옹,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든,극장판 바이올렛 에버가든
2,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,가나의 혼인잔치: 언약,반지의 제왕: 왕의 귀환,패왕별희 디 오리지널,패왕별희 디 오리지널,패왕별희 디 오리지널,패왕별희 디 오리지널,패왕별희 디 오리지널,패왕별희 디 오리지널,부활: 그 증거,부활: 그 증거,레옹,레옹,레옹,다시 태어나도 우리,다시 태어나도 우리,레옹,레옹,레옹
3,반지의 제왕: 왕의 귀환,소년시절의 너,소년시절의 너,소년시절의 너,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,죽은 시인의 사회,극장판 귀멸의 칼날: 무한열차편,극장판 귀멸의 칼날: 무한열차편,극장판 귀멸의 칼날: 무한열차편,극장판 귀멸의 칼날: 무한열차편,극장판 귀멸의 칼날: 무한열차편,극장판 귀멸의 칼날: 무한열차편,패왕별희 디 오리지널,패왕별희 디 오리지널,부활: 그 증거,부활: 그 증거,부활: 그 증거,피아니스트,부활: 그 증거,다시 태어나도 우리,다시 태어나도 우리,다시 태어나도 우리
4,죽은 시인의 사회,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,반지의 제왕: 왕의 귀환,죽은 시인의 사회,죽은 시인의 사회,죽은 시인의 사회,죽은 시인의 사회,죽은 시인의 사회,죽은 시인의 사회,죽은 시인의 사회,죽은 시인의 사회,죽은 시인의 사회,반지의 제왕: 두 개의 탑,소울,소울,소울,소울,소울,소울,극장판 귀멸의 칼날: 무한열차편,극장판 귀멸의 칼날: 무한열차편,피아니스트,피아니스트,피아니스트,패왕별희 디 오리지널,피아니스트,부활: 그 증거,부활: 그 증거,부활: 그 증거
5,반지의 제왕: 두 개의 탑,죽은 시인의 사회,죽은 시인의 사회,죽은 시인의 사회,반지의 제왕: 두 개의 탑,반지의 제왕: 두 개의 탑,반지의 제왕: 두 개의 탑,반지의 제왕: 두 개의 탑,반지의 제왕: 두 개의 탑,반지의 제왕: 두 개의 탑,반지의 제왕: 두 개의 탑,반지의 제왕: 두 개의 탑,반지의 제왕: 두 개의 탑,패왕별희 디 오리지널,천녀유혼,천녀유혼,천녀유혼,천녀유혼,자산어보,자산어보,소울,소울,패왕별희 디 오리지널,패왕별희 디 오리지널,패왕별희 디 오리지널,극장판 귀멸의 칼날: 무한열차편,패왕별희 디 오리지널,피아니스트,피아니스트,피아니스트
